In [1]:
import pandas as pd
import numpy as np
#pd.set_option('precision', 2)

from datetime import timedelta
from datetime import datetime
import dateutil.parser as parser
#import dateparser
from openpyxl import Workbook
from openpyxl import load_workbook
#!pip install openpyxl --upgrade
import xlrd

In [2]:
#############################################################

######### Agrega la fecha del dia a procesar
d= '8123'   
u= '56959'

In [3]:
# set the directory path
path = f'C:/Users/{u}/Downloads/Cierre diario 2023/Ago/{d}/'  

In [4]:
# 2) CASH RECEIPTS REPORT
file = "CashReceiptsReport.xls"

#ref_workbook= openpyxl.reader.excel.load_workbook(path + file)

#wb = load_workbook(r'C:\Users\Leja\Downloads\Cierre diario 2023\Junio\6223\CashReceiptsReport.xls')

dfc = pd.read_excel(path + file, engine='xlrd',  dtype={'Policy Number': 'str'}).fillna('')

#print(dfc)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
df2 = dfc.iloc[4:, :]
#print(df2)
#df2.dtypes

df3=df2.rename(columns={"Cash Receipts": "TYPE",  "Unnamed: 1": "Payer", "Unnamed: 4": "Receipt #", "Unnamed: 6": "Check #", 
                        "Unnamed: 8": "Comment", "Unnamed: 10": "Status", 
                        "Unnamed: 12": "Receipt Date", "Unnamed: 14": "Receipt Amount", "Unnamed: 16":"Entered By"})

df4= df3[["TYPE", "Payer", "Receipt #", "Check #", "Comment", "Status", "Receipt Date", "Receipt Amount", "Entered By"]]
df4['TYPE'] = df4['TYPE'].replace('', np.nan).ffill() #rellena filas siguientes con último dato
df4 = df4[(df4['Status'].str.len() > 0) ]
df4 = df4[(df4['Payer'].str.len() > 0) ]

crr=df4

#CONVERTIR FINES DE SEMANA A VIERNES

crr['dia'] = pd.to_datetime(crr['Receipt Date']).dt.dayofweek
# Monday=0, Sunday=6
#restar días

def dias(df):
    if df['dia'] == 6:
        return (df['Receipt Date']- pd.Timedelta(days = 2))
    elif df['dia'] == 5:
        return (df['Receipt Date']- pd.Timedelta(days = 1))
    elif df['dia'] == 0:
        return (df['Receipt Date']- pd.Timedelta(days = 3))    
    else:
        return df['Receipt Date']

crr['Receipt Date_2'] = crr.apply(dias, axis=1) 

C:\Users\56959\AppData\Local\Temp\ipykernel_10612\141138170.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['TYPE'] = df4['TYPE'].replace('', np.nan).ffill() #rellena filas siguientes con último dato


In [6]:
#print(crr)

In [7]:
crr['Receipt Date'] = pd.to_datetime(crr['Receipt Date']).dt.date
crr['Receipt Date_2'] = pd.to_datetime(crr['Receipt Date_2']).dt.date

crr.sort_values(by=['Receipt Date_2'])
crr['RD'] =crr['Receipt Date_2'].astype(str)
date = crr['RD'].unique().tolist()

#print(date)
#crr.dtypes
print (crr)

            TYPE                              Payer Receipt # Check #  \
7           Cash                 Alejandro Martines    031193           
11          Cash                 Brayan Navas Funes    031188           
15          Cash              Brenda Gamez Guevara     031186           
19          Cash       Carmen Darisa Reina Anchecta    031185           
23          Cash       Diana Xiomar Flores Mcdermot    031189           
27          Cash                      JKL GLASS LLC    031183           
31          Cash                    Maria Diaz Diaz    031184           
35          Cash                RIGO MARQUEZ ORTEGA    031200           
39          Cash                      Yesenia Lopez    031187           
46         Check                PERCAM PAINTING LLC    031203           
48         Check                PERCAM PAINTING LLC    031204           
55   CreditOrEFT      Amber Michelle Mota Dominguez    031208           
59   CreditOrEFT                 AMIGO AUTO ONE LLC

In [8]:
# 3) DIRECT BILL CASH REPORT

file = "DirectBillCashReport.xls"

dfb = pd.read_excel(path + file, engine='xlrd',  dtype={'Policy Number': 'str'}).fillna('')


#dfb = pd.read_excel("DirectBillCashReport.xls")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
df2 = pd.DataFrame(dfb)
df3 = df2.iloc[10:, :]

df3=df3.rename(columns={"Unnamed: 1": "Receipt #", "Unnamed: 2": "Payer", "Unnamed: 4":'Comment/Policy #', "Unnamed: 5": "Receipt Type", "Unnamed: 6": "Receipt Date", 
                        "Unnamed: 7": "Deposit Ref. #", "Unnamed: 8":"Receipt Amount", "Unnamed: 10": "Amount due Carrier", "Unnamed: 11": "Fees", "Unnamed: 13": "Payment To", 
                        "Unnamed: 15": "Check Ref. #", "Unnamed: 16": "Check Date", "Unnamed: 17": "Check Amount", "Unnamed: 18": "Balance", "Unnamed: 20": "Entered By"})

df4= df3[["Receipt #", "Payer", 'Comment/Policy #', "Receipt Type", "Receipt Date", "Deposit Ref. #", "Receipt Amount", "Amount due Carrier", 
          "Fees", "Payment To", "Entered By"]]

df4 = df4[(df4['Payer'].str.len() > 0) ]
#print(df4)
#df4.dtypes
dbc=df4

#CONVERTIR FINES DE SEMANA A VIERNES

dbc['dia'] = pd.to_datetime(dbc['Receipt Date']).dt.dayofweek
# Monday=0, Sunday=6
#restar días

def dias(df):
    if df['dia'] == 6:
        return (df['Receipt Date']- pd.Timedelta(days = 2))
    elif df['dia'] == 5:
        return (df['Receipt Date']- pd.Timedelta(days = 1))
    elif df['dia'] == 0:
        return (df['Receipt Date']- pd.Timedelta(days = 3))
    else:
        return df['Receipt Date']

dbc['Receipt Date_2'] = dbc.apply(dias, axis=1)  

dbc['Receipt Date'] = pd.to_datetime(dbc['Receipt Date']).dt.date
dbc['Receipt Date_2'] = pd.to_datetime(dbc['Receipt Date_2']).dt.date
dbc.sort_values(by=['Receipt Date_2'])
dbc['RD'] =dbc['Receipt Date_2'].astype(str)

In [10]:
# 4) Invoice Line Item Detail EZLynx
#dfi = pd.read_excel("InvoiceLineItemDetail.xlsx")

file = "InvoiceLineItemDetail.xls"

dfi = pd.read_excel(path + file, engine='xlrd', dtype={'Policy Number': 'str'}).fillna('')



WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [11]:
df2 = pd.DataFrame(dfi)
#df2.columns = df2.iloc[4]
df3 = df2.iloc[5:, :]

df3=df3.rename(columns={"Unnamed: 1": "Line Item Type", "Unnamed: 6":'Payer', "Unnamed: 14": "Invoice No.", "Unnamed: 17": "Receipt Date", "Unnamed: 22": "Amount"})
df4= df3[["Line Item Type", 'Payer', "Invoice No.", "Receipt Date", "Amount"]]
df4 = df4[(df4['Payer'].str.len() > 0) ]

ili=df4

#ili.dtypes
#print(ili)

#CONVERTIR FINES DE SEMANA A VIERNES

ili['dia'] = pd.to_datetime(ili['Receipt Date']).dt.dayofweek
# Monday=0, Sunday=6
#restar días

def dias(df):
    if df['dia'] == 6:
        return (df['Receipt Date']- pd.Timedelta(days = 2))
    elif df['dia'] == 5:
        return (df['Receipt Date']- pd.Timedelta(days = 1))
    elif df['dia'] == 0:
        return (df['Receipt Date']- pd.Timedelta(days = 3))
    else:
        return df['Receipt Date']

ili['Receipt Date_2'] = ili.apply(dias, axis=1)  

ili['Receipt Date'] = pd.to_datetime(ili['Receipt Date']).dt.date
ili['Receipt Date_2'] = pd.to_datetime(ili['Receipt Date_2']).dt.date
ili.sort_values(by=['Receipt Date_2'])
ili['RD'] =ili['Receipt Date_2'].astype(str)

In [12]:
#5) BLACK STONE
#CREDIT

file = f'BS Credit {d}.xlsx'

credit = pd.read_excel(path + file, engine='openpyxl', dtype={'Policy Number': 'str'}).fillna('')

#credit = pd.read_excel(f'BS Credit {d}.xlsx')
credit['TYPE']="CreditOrEFT"
credit.dtypes

BS= credit[['Ref #',	'Cust ID',	'Inv #',	'Date',	'Result',	'Payment Type',	'Account Type',	'Account #',	'Name',	'TYPE',	'Status',	'Approval Code',	'Auth Amt',	'Capture Amt',	'Tip Amt',	'User']]

In [13]:
#BLACK STONE
#CHECK

file = f'BS Check {d}.xlsx'

check = pd.read_excel(path + file, engine='openpyxl', dtype={'Policy Number': 'str'}).fillna('')

check['TYPE']="Check"

check=check.rename(columns={'Username': "User", 'Amount':'Auth Amt'})

#print(check)
frames = [credit, check]
BS = pd.concat(frames)


In [14]:
#NEW BUSINESS FEES

file = 'PolicyTransaction_Master.xlsx'

nb = pd.read_excel(path + file, engine='openpyxl', dtype={'Policy Number': 'str'}).fillna('')


#nb = nb.loc[~((nb['Line of Business'].isin(['Genl Liability', 'Workers comp'])) | (nb['Transaction Type']=='Renewal')), :] 

nb['drop'] = np.where((nb['Line of Business'].isin(['Genl Liability', 'Workers comp'])) & (nb['Transaction Type']=='Renewal') ,'1',
                      np.where((nb['Transaction Type']!='Renewal') ,'1','0'))

nb = nb.loc[nb['drop']=='1' ]

nb['Address'] = nb['Address - City'] + " " + nb['Address - Line 1']

nb_= nb[['Address', 'Account Name']]

#print(nb)

nb_ad = pd.pivot_table(nb, values='Policy Number', index='Account Name', aggfunc='count', fill_value=0).fillna(0)
nb_add = pd.DataFrame(nb_ad.to_records())
#print(nb_add)

#staate
nb_add= pd.merge(nb_add, nb_, how="left", on='Account Name').drop_duplicates(subset=['Account Name'], keep='last')



file = 'InvoiceLineItemDetail Mes.xls'

df2 = pd.read_excel(path + file, engine='xlrd', dtype={'Policy Number': 'str'}).fillna('')

df2 = pd.DataFrame(df2)
#df2.columns = df2.iloc[4]
df3 = df2.iloc[5:, :]

df3=df3.rename(columns={"Unnamed: 1": "Line Item Type", "Unnamed: 6":'Payer', "Unnamed: 14": "Invoice No.", "Unnamed: 17": "Receipt Date", "Unnamed: 22": "Amount"})
df4= df3[["Line Item Type", 'Payer', "Invoice No.", "Receipt Date", "Amount"]]
df4 = df4[(df4['Payer'].str.len() > 0) ]

ili_mes=df4

ili_ad = pd.pivot_table(ili_mes, values='Amount', index='Payer', columns='Line Item Type', aggfunc='sum', margins=True, margins_name='Total').fillna(0)
ili_add = pd.DataFrame(ili_ad.to_records()).fillna(False)


comnt= crr[['Payer', 'Comment']].fillna('')

#comnt['mar']= 'x'
#ili_add= pd.merge(ili_add, comnt, how="left", on='Payer')


for i in ili_add['Payer'].to_list():
    comnt.loc[comnt['Payer'].astype(str).str.contains(i), 'Payer' ] = i

ili_add= pd.merge(ili_add, comnt, how="left", on='Payer')

print(ili_add)


#print(ili_add)

for i in ili_add['Payer'].to_list():
    nb_add.loc[nb_add['Account Name'].astype(str).str.contains(i), 'Payer' ] = i

nb_fe= pd.merge(nb_add, ili_add, how="left", on='Payer')

id_nb= nb [['Applicant ID',	'Account Name',	'Assigned Producer']]

#print(id_nb)

nb_fee= pd.merge(nb_fe, id_nb, how="left", on='Account Name').drop_duplicates(subset=['Account Name'], keep='last')


print(nb_fee)

#rint(result)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
                                Payer  Cash Fee  Commercial Admin Fee  \
0               ALBARRAN CONSTRUCTION       0.0                   0.0   
1    ACE ROOFING AND CONSTRUCTION LLC       5.0                   0.0   
2                ADAN LOPEZ HERNANDEZ       5.0                   0.0   
3       AGILE WOLF BEST SERVICES CORP       0.0                   0.0   
4                  ALFONSO BAEZ PEREZ       0.0                 100.0   
..                                ...       ...                   ...   
293               Yessenia Ayala Cruz       0.0                   0.0   
294             Yobani Garcia Marquez       5.0                   0.0   
295                     Yohania Lopez       0.0                   0.0   
296              irismar schoonewolff       0.0                   0.0   
297                             Total     145.0                4008.0   

     Commercial General Liability Fee  Commercial 

In [15]:
nb_fee_1 = pd.pivot_table(nb_fee, values='Total', index='Assigned Producer', aggfunc='sum' ).fillna(0)
nb_fee_2 = pd.pivot_table(nb, values='Policy Number', index='Assigned Producer', aggfunc='count').fillna(0)

nb_fee_ad= nb_fee_2.join(nb_fee_1)

nb_fee_ad= pd.DataFrame(nb_fee_ad.to_records()).append(nb_fee_ad.sum().rename('Total')).fillna(0)


print(nb_fee_ad)  

                         Assigned Producer  Policy Number   Total
0                        Alcazar, Isabella            1.0     0.0
1                         Arroyave, Steven            1.0     0.0
2      Assistant Principal, Joe and Carlos            2.0    80.0
3                           Caruso, Rosana            1.0     0.0
4                       Cervantes, Giselle            3.0   150.0
5                           Da Costa, Joel            1.0    50.0
6                      Dacosta, Flor Betsy            4.0   200.0
7                        DeLaRosa, Yadiris            1.0    25.0
8                     Dossantos, Francisco            2.0     0.0
9                         Freay, Alejandro            3.0     0.0
10                         Garcia, Genesis            2.0   100.0
11                            Karen, Karen            3.0   150.0
12                             Lora, Luisa            1.0    50.0
13                 Renewal Team , valeria             3.0   150.0
14        

C:\Users\56959\AppData\Local\Temp\ipykernel_10612\4060718057.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nb_fee_ad= pd.DataFrame(nb_fee_ad.to_records()).append(nb_fee_ad.sum().rename('Total')).fillna(0)


In [16]:
BS= BS[['Ref #',	'Cust ID',	'Inv #',	'Date',	'Result',	'Payment Type',	'Account Type',	'Account #',	'Name',	'TYPE',	'Status',	'Approval Code',	'Auth Amt',	'Capture Amt',	'Tip Amt',	'User']]

BS.drop(index=BS[BS['Ref #'] == 'Ref #'].index, inplace=True)

#print(BS)
#BS.dtypes

In [17]:
#CONVERTIR FINES DE SEMANA A VIERNES

BS['dia'] = pd.to_datetime(BS['Date']).dt.dayofweek
# Monday=0, Sunday=6
#restar días

def dias(df):
    if df['dia'] == 6:
        return (df['Date']- pd.Timedelta(days = 2))
    elif df['dia'] == 5:
        return (df['Date']- pd.Timedelta(days = 1))
    elif df['dia'] == 0:
        return (df['Date']- pd.Timedelta(days = 3))
    else:
        return df['Date']

BS['Date_2'] = BS.apply(dias, axis=1)  

BS['Date'] = pd.to_datetime(BS['Date']).dt.date
BS['Date_2'] = pd.to_datetime(BS['Date_2']).dt.date
BS.sort_values(by=['Date_2'])
BS['RD'] =BS['Date_2'].astype(str)


In [18]:
uniques = crr.RD.unique()

for i in uniques:
    with pd.ExcelWriter(f'Cierre diario {i}.xlsx') as writer:
    ######## CASH RECEIPTS REPORT ###############
        crr[crr.RD == i].to_excel(writer, index=None, sheet_name='CRR')

    ######## DIRECT BILL CASH REPORT #############
        dbc[dbc.RD == i].to_excel(writer, index=None, sheet_name='DBC')

    ######## INLINE INVOICE #################
        ili[ili.RD == i].to_excel(writer, index=None, sheet_name='ILI')

    ######## BLACK STONE ###############
        BS[BS.RD == i].to_excel(writer, index=None, sheet_name='BS') 

    ######### RESUMEN FORMA DE PAGO###############
        crr2=crr[crr.RD == i].pivot_table(index='Entered By', columns=['TYPE'], values= ['Receipt Amount'], aggfunc='sum', margins=True, margins_name='Total').fillna(0)
        crr3 = pd.DataFrame(crr2.to_records())
        crr3.to_excel(writer, index=None, sheet_name='CRR resumen')

    ######### RESUMEN FORMA DE PAGO###############
        BS1=BS[BS.RD == i].reset_index().pivot_table(values='Auth Amt', index='User', columns='TYPE', aggfunc='sum').fillna(0)
        BS_1 = pd.DataFrame(BS1.to_records())
        BS_1.to_excel(writer, index=None, sheet_name='BS resumen')

    ######### RESUMEN NEW BUSINESS ###############
    #BS1=BS[BS.RD == i].reset_index().pivot_table(values='Auth Amt', index='User', columns='TYPE', aggfunc='sum').fillna(0)
    #BS_1 = pd.DataFrame(BS1.to_records())
        nb.to_excel(writer, index=None, sheet_name='New Business')
        nb_fee.to_excel(writer, index=None, sheet_name='NB FEES')
        nb_fee_ad.to_excel(writer, index=None, sheet_name='Fees resumen')
